Demo is running on Python 3.7.7

In [1]:
import numpy as np

In [2]:
from doe_tool import doe_tool

Suppose we have a simple model of single interaction dynamics

In [3]:
model = '''
$X->Y; beta*X;
Y-> ; alpha*Y;

X = 1; Y = 0;
beta = 1;
alpha = 0.1;

'''

The following command initializes the model that we want to analyze

In [4]:
g = doe_tool(model)

Error: Could not open stream: File not found


Once the model is loaded, a simulator coroutine is created and calls to it are
performed internally like so:

In [5]:
test = g.simulate({'alpha': .1, 'beta': 1})
test

        time,      [Y]
 [[        0,        0],
  [ 0.505051, 0.492509],
  [   1.0101,  0.96076],
  [  1.51515,  1.40595],
  [   2.0202,  1.82921],
  [  2.52525,  2.23163],
  [   3.0303,  2.61423],
  [  3.53535,  2.97798],
  [   4.0404,  3.32382],
  [  4.54545,  3.65263],
  [  5.05051,  3.96525],
  [  5.55556,  4.26247],
  [  6.06061,  4.54505],
  [  6.56566,  4.81371],
  [  7.07071,  5.06914],
  [  7.57576,  5.31199],
  [  8.08081,  5.54288],
  [  8.58586,  5.76239],
  [  9.09091,  5.97109],
  [  9.59596,  6.16952],
  [   10.101,  6.35817],
  [  10.6061,  6.53753],
  [  11.1111,  6.70806],
  [  11.6162,  6.87019],
  [  12.1212,  7.02434],
  [  12.6263,  7.17089],
  [  13.1313,  7.31023],
  [  13.6364,   7.4427],
  [  14.1414,  7.56865],
  [  14.6465,  7.68839],
  [  15.1515,  7.80224],
  [  15.6566,  7.91048],
  [  16.1616,  8.01338],
  [  16.6667,  8.11122],
  [  17.1717,  8.20424],
  [  17.6768,  8.29268],
  [  18.1818,  8.37676],
  [  18.6869,   8.4567],
  [  19.1919,   8.5327],
  

Extracting other values form the model that do not require simulation (such as Eigen values) is in a To Do list, but will similarly be calls to the coroutine

Post processing of Time Series data is expected to vary between cases, so the program wraps
any normal function into a coroutine:

In [6]:
def post_processing(data):
    # Example of finding response time of a model
    # extract last value in dataset, divide by 2 and find
    # location of closest value in dataset
    halfmax = (data['[Y]'][-1])/2
    return data['time'][np.argmin(abs(data['[Y]'] - halfmax))]

In [7]:
g.load_post_processor(post_processing)

Post-processed information is then evalueated against a certain range of values too see if the requirement is met. In here, we call them conditionals

In [8]:
# Conditionals setup
g.set_conditional('boundary', 0, 2)

At this point the setup is finished. There are 2 options for model analysis present: scanning and optimizing.

We will use the following parameters for our Grid Search

In [9]:
grid_search_parameters = (
    ('alpha', [.01, .1, 1, 10, 100] ),
    ('beta', range(1, 5))
)
# The above specifies what parameter you want to vary and the range/list of values to check

### optimize

Faster option; does not save any runs except for the first one that satified the conditional

In [10]:
g.optimize_demo('grid', grid_search_parameters)

{'alpha': 1, 'beta': 1}

To check that we are indeed within specified boundary of (0,2)

In [11]:
g.send_post_processor(g.simulate({'alpha': 1, 'beta': 1}))

0.5050505050505051

### scan

Unlike optimize, scan goes through all listed values for our grid search and saves every value

In [12]:
g.scan('grid', grid_search_parameters) #load in parameters of interest and start scanning

Getting the intermediate runs, or the complete scan is possible using the following command

In [13]:
specs, sims, results = g.get_scan()

In [14]:
print((results[0], results[5]))
print((specs[0], specs[5]))

(21.71717171717172, 7.070707070707071)
({'alpha': 0.01, 'beta': 1}, {'alpha': 0.1, 'beta': 2})
